In [15]:
from langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from typing import Annotated,Sequence,TypedDict,Literal
from langchain_core.messages import BaseMessage,HumanMessage
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.vectorstores.chroma import Chroma
from langchain_groq import ChatGroq
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub
from langchain.pydantic_v1 import BaseModel,Field
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode,tools_condition
from langchain_core.tools.retriever import create_retriever_tool
from langchain.document_loaders import WebBaseLoader
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [3]:

urls = [    'https://dahfood.com/DAHFOOD-Sultana-raisin/',
                    'https://dahfood.com/DAHFOOD-Red-pistachio-kernel/',
                    'https://dahfood.com/DAHFOOD-Media/',
                    'https://dahfood.com/DAHFOOD-Dates-Catalogues/',
                    'https://dahfood.com/Red-Apple/',
                    'https://dahfood.com/DAHFOOD-Kabkab-dates/',
                    'https://dahfood.com/DAHFOOD-Fandoghi/',
                    'https://dahfood.com/DAHFOOD-Golden-Raisin/',
                    'https://dahfood.com/DAHFOOD-Contact-us/',
                    'https://dahfood.com/DAHFOOD-Moringa-leaf/',
                    'https://dahfood.com/documents/',
                    'https://dahfood.com/Date-seed-kernel/',
                    'https://dahfood.com/',
                    'https://dahfood.com/DAHFOOD-Moringa-powder/',
                    'https://dahfood.com/DAHFOOD-About-us/',
                    'https://dahfood.com/Date-syrup/',
                    'https://dahfood.com/DAHFOOD-Pistachio-Catalogues/',
                    'https://dahfood.com/DAHFOOD-Dried-fruit-Catalogues/',
                    'https://dahfood.com/DAHFOOD-Golden-Kashmari-raisin/',
                    'https://dahfood.com/DAHFOOD-Sun-dried-raisin/',
                    'https://dahfood.com/DAHFOOD-Sliced-dates/',
                    'https://dahfood.com/DAHFOOD-Packing-Label/',
                    'https://dahfood.com/DAHFOOD-Mavis-raisins/',
                    'https://dahfood.com/DAHFOOD-Quality-Control-Guarantee/',
                    'https://dahfood.com/DAHFOOD-Moringa/',
                    'https://dahfood.com/DAHFOOD-Laboratory-and-Technical-Data/',
                    'https://dahfood.com/DAHFOOD-Green-kashmiri-raisin/',
                    'https://dahfood.com/DAHFOOD-Jumbo/',
                    'https://dahfood.com/DAHFOOD-Piarom-dates/',
                    'https://dahfood.com/DAHFOOD-Pistachio/',
                    'https://dahfood.com/#wz-section-wzs297',
                    'https://dahfood.com/DAHFOOD-Dates/',
                    'https://dahfood.com/DAHFOOD-Raisin-Catalogues/',
                    'https://dahfood.com/DAHFOOD-Moringa-oil/',
                    'https://dahfood.com/DAHFOOD-Moringa-Catalogues/',
                    'https://dahfood.com/DAHFOOD-Date-paste/',
                    'https://dahfood.com/DAHFOOD-Mazafati-dates/',
                    'https://dahfood.com/DAHFOOD-Risins/',
                    'https://dahfood.com/DAHFOOD-Rabbi-dates/',
                    'https://dahfood.com/DAHFOOD-Sayer-dates/',
                    'https://dahfood.com/DAHFOOD-Zahedi-Dates/',
                    'https://dahfood.com/DAHFOOD-Green-Peeled-Pistachio-Kernel/',
                    'https://dahfood.com/DAHFOOD-FAQ/',
                    'https://dahfood.com/DAHFOOD-Ahmad-Aghaei/',
                    'https://dahfood.com/DAHFOOD-Akbari/',
                ]

docs = [WebBaseLoader(url).load() for url in urls]
doc_slid = [item for doc in docs for item in doc]

character_chunk = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=200,chunk_overlap=50
)

text_spliter = character_chunk.split_documents(doc_slid)

vector_store = Chroma.from_documents(
    documents=text_spliter,
    embedding=embedding
)
retriver = vector_store.as_retriever()

In [4]:

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [5]:
class GradScore(BaseModel):
    """based on relavance of 'yes' or 'no'"""
    binary_score: str = Field(description="give the answer based on document are related to question in 'yes' or 'no'")

In [6]:

retriever_tool=create_retriever_tool(
    retriver,
    "retrieve_blog_posts",
    "Search and return information about Lilian Weng blog posts on LLM agents, prompt engineering, and adversarial attacks on LLMs.You are a specialized assistant. Use the 'retriever_tool' **only** when the query explicitly relates to LangChain blog data. For all other queries, respond directly without using any tool. For simple queries like 'hi', 'hello', or 'how are you', provide a normal response.",
    )

In [7]:
tool = [retriever_tool]

In [8]:
tool_node = ToolNode([retriever_tool])

In [10]:

def grade_documents(state:AgentState)->Literal["Output_Generator", "Query_Rewriter"]:
    llm_with_structure_op=llm.with_structured_output(GradScore)
    
    prompt=PromptTemplate(
        template="""You are a grader deciding if a document is relevant to a user’s question.
                    Here is the document: {context}
                    Here is the user’s question: {question}
                    If the document talks about or contains information related to the user’s question, mark it as relevant. 
                    Give a 'yes' or 'no' answer to show if the document is relevant to the question.""",
                    input_variables=["context", "question"]
                    )
    chain = prompt | llm_with_structure_op
    
    messages = state["messages"]
    last_message = messages[-1]
    question = messages[0].content
    docs = last_message.content
    scored_result = chain.invoke({"question": question, "context": docs})
    score = scored_result.binary_score

    if score == "yes":
        print("---DECISION: DOCS RELEVANT---")
        return "generator" #this should be a node name
    else:
        print("---DECISION: DOCS NOT RELEVANT---")
        return "rewriter" #this should be a node name

In [11]:
def generator(state: AgentState):
    print("---GENERATE---")
    messages = state["messages"]

    question = messages[0].content
    
    last_message = messages[-1]
    docs = last_message.content
    
    prompt = hub.pull("rlm/rag-prompt")
    
    rag_chain = prompt | llm

    response = rag_chain.invoke({"context": docs, "question": question})
    print(f"this is my response:{response}")
    
    return {"messages": [response]}


In [12]:
def rewrite(state:AgentState):
    print("---TRANSFORM QUERY---")
    messages = state["messages"]
    question = messages[0].content
    
    message = [HumanMessage(content=f"""Look at the input and try to reason about the underlying semantic intent or meaning. 
                    Here is the initial question: {question} 
                    Formulate an improved question: """)
       ]
    response = llm.invoke(message)
    return {"messages": [response]}

In [25]:
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_community.tools.tavily_search import TavilyAnswer

In [ ]:
def reserch_agent(state):
    question = state['messages'][-1]
    
    tools = [TavilyAnswer(name="Intermediate Answer")]
    
    prompt = hub.pull("hwchase17/self-ask-with-search")
    
    agent = create_self_ask_with_search_agent(llm, tools, prompt)
    
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) 
    
    response = agent_executor.invoke({"input": question})
    
    return {'messages':[response]}

In [27]:
reserch_agent({'messages':["what is export import business"]})

c:\Users\ADMIN\anaconda3\envs\jay\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...


KeyError: "Input to PromptTemplate is missing variables {'tool_names', 'tools'}.  Expected: ['agent_scratchpad', 'input', 'tool_names', 'tools'] Received: ['input', 'intermediate_steps', 'agent_scratchpad', 'chat_history']\nNote: if you intended {tool_names} to be part of the string and not a variable, please escape it with double curly braces like: '{{tool_names}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

In [ ]:

workflow=StateGraph(AgentState)
workflow.add_node("Vector_Retriever", tool_node) 
workflow.add_node("Query_Rewriter", rewrite) 
workflow.add_node("Output_Generator", generator)
workflow.add_node("research_agent",reserch_agent)

workflow.add_edge(START,"Vector_Retriever")
workflow.add_conditional_edges("Vector_Retriever",
                            grade_documents,
                            {"generator": "Output_Generator",
                            "rewriter": "Query_Rewriter"
                            }
                            )
workflow.add_edge("Output_Generator", END)
workflow.add_edge("Query_Rewriter", "research_agent")
workflow.add_edge("research_agent",END)
app=workflow.compile()
app

ValueError: Found edge ending at unknown node `My_Ai_Assistant`